AI-Powered Diet Planner 🍎 (ML + Nutrition Science)

In [2]:
pip install pandas numpy requests scikit-learn flask streamlit 

In [3]:
import pandas as pd

# Load dataset (Ensure the correct file path)
df = pd.read_csv('C:/USDA.csv', encoding='utf-8')

# Strip spaces from column names
df.columns = df.columns.str.strip()

# Print available columns to verify names
print("Available columns:", df.columns.tolist())


Available columns: ['ID', 'Description', 'Calories', 'Protein', 'TotalFat', 'Carbohydrate', 'Sodium', 'SaturatedFat', 'Cholesterol', 'Sugar', 'Calcium', 'Iron', 'Potassium', 'VitaminC', 'VitaminE', 'VitaminD']


In [4]:
# Rename columns based on actual dataset output
df.rename(columns={
    'Carbohydrate': 'Carbs',  # Adjust this name if needed
    'TotalFat': 'Fats'        # Adjust this name if needed
}, inplace=True)

# Drop missing values
df = df.dropna(subset=['Calories', 'Protein', 'Carbs', 'Fats'])

print("✅ Data cleaned successfully!")





✅ Data cleaned successfully!


In [5]:
if 'Category' not in df.columns:
    if 'Food' in df.columns:
        df['Category'] = df['Food'].apply(lambda x: 'Fruit' if isinstance(x, str) and 'apple' in x.lower() else 'Other')
    else:
        df['Category'] = 'Unknown'  # Default if 'Food' column is missing

print("✅ 'Category' column created successfully!")


✅ 'Category' column created successfully!


In [6]:
from sklearn.preprocessing import LabelEncoder

# Encode category labels
encoder = LabelEncoder()
df['Category'] = encoder.fit_transform(df['Category'])

print("✅ 'Category' encoding completed successfully!")


✅ 'Category' encoding completed successfully!


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Encode category labels
encoder = LabelEncoder()
df['Category'] = encoder.fit_transform(df['Category'])

# Features & Target
X = df[['Calories', 'Protein', 'Carbs', 'Fats']]
y = df['Category']

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate Model
accuracy = model.score(X_test, y_test)
print(f"✅ Model Accuracy: {accuracy:.2f}")


✅ Model Accuracy: 1.00


In [8]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.2f}")

# Show detailed classification report
print(classification_report(y_test, y_pred))


✅ Model Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1412

    accuracy                           1.00      1412
   macro avg       1.00      1.00      1.00      1412
weighted avg       1.00      1.00      1.00      1412



In [9]:
# Example: Predict category for a new food item
new_data = pd.DataFrame({'Calories': [100], 'Protein': [5], 'Carbs': [20], 'Fats': [2]})
predicted_category = model.predict(new_data)

# Decode category if using LabelEncoder
category_name = encoder.inverse_transform(predicted_category)
print(f"🥗 Predicted Food Category: {category_name[0]}")


🥗 Predicted Food Category: 0


In [10]:
import joblib

# Save model and encoder
joblib.dump(model, 'diet_planner_model.pkl')
joblib.dump(encoder, 'category_encoder.pkl')
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [11]:
pip install flask


Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd

app = Flask(__name__)

# Load the trained model and encoder
model = joblib.load('diet_planner_model.pkl')
encoder = joblib.load('category_encoder.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get JSON data from request
        data = request.json
        
        # Convert input data to DataFrame
        input_df = pd.DataFrame([data])
        
        # Predict category
        prediction = model.predict(input_df)
        
        # Decode category label
        category_name = encoder.inverse_transform(prediction)
        
        return jsonify({'category': category_name[0]})
    
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    nutrients = [[data['Calories'], data['Protein'], data['Carbs'], data['Fats']]]
    prediction = model.predict(nutrients)
    category = encoder.inverse_transform(prediction)
    return jsonify({'Category': category[0]})

# Run Flask (Note: This will stop Jupyter execution)
app.run(port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2025 00:08:41] "GET / HTTP/1.1" 404 -


In [3]:
curl -X POST http://127.0.0.1:5000/predict -H "Content-Type: application/json" -d '{"Calories": 250, "Protein": 10, "Carbs": 30, "Fats": 5}'


SyntaxError: invalid syntax (<ipython-input-3-ce63feeb4a93>, line 1)

In [4]:
@app.route('/')
def home():
    return "Welcome to the AI Diet Planner API! Use /predict to get food category predictions."


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Flask is running!"

if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2025 00:11:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2025 00:11:56] "GET /predict HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2025 00:11:57] "GET /predict HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2025 00:11:58] "GET /predict HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2025 00:11:58] "GET /predict HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2025 00:12:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2025 00:12:06] "GET / HTTP/1.1" 200 -
